In this script, I try to use RL to solve the knapsack problem.
This should not be an exciting project but just a warm up (for me at least) to understand hands-on RL, RNN, embeddings and such.

The project starts with:
- a nn to move from 2 features per object to 8
- a LTSM with an internal state composed by 5 features
- its output is fed to a fully connected layer that from 5 feature expands to the number of objects
- a softmax decides which element to choose
- If you chose objects whose weight is more than 1, the reward is -100

Still to implement:
- attention mechanism
- mask to not choose 'wrong' objects

As always, we start by importing useful libraries and defiening some boundary conditions

In [34]:
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import torch 
import random
import itertools    

%matplotlib inline

random.seed(1234)
prize_min = 0
prize_max = 100

Then we define the object class

In [35]:
class ObjectCass:
    
    Prize = None
    Weight = None
    
    def __init__(self, reward, weight):
        
        self.Prize = reward
        self.Weight = weight

Now we define the function to generate new instances.

In [36]:
def GenerateANewInstance(num_objs):
    
    Objects = []
    for i in range(len(num_objs)):
        prize = round(random.uniform(prize_min, prize_max), 0)
        weight = round(random.uniform(0, 1), 2)
        obj = ObjectCass(prize,weight)
        Objects.append(obj)
    Object.append(ObjectCass(0,0)) # fake object representing that you don't pick up anything
                                   # i.e. you stop picking up stuff
    
    return Objects

Now we define a very general agent

In [37]:
class AgentClass:
    def __init__(self):
        self.total_reward = 0.0

    def step(self, env):
        
        current_obs = env.get_observation() #this could be rmeoved I think
        actions = env.get_actions()
        reward = env.action(random.choice(actions))
        self.total_reward += reward
        
        return action
Agent = AgentClass()

Now I try to create a shallow NN. from the features of the input (which are 2), I want to output 10 features (whith one hidden layer of size 10 as well).

In [55]:
class EmbeddingPlusLSTM(torch.nn.Module):
    
    def __init__(self, num_input, num_embeddings, num_h, num_output, batch_size):    
        
        super(EmbeddingPlusLSTM, self).__init__() # this is not really clear to me
        
        self.fc1 = torch.nn.Linear(num_input, num_embeddings)
        self.fc2 = torch.nn.Linear(num_embeddings, num_embeddings)
        self.lstm= torch.nn.LSTM(num_embeddings,  num_h)
        self.fc3 = torch.nn.Linear(num_h, num_output)
        
        # here I set the first internal state of the RNN.
        # I set it here and not in forwards otherwise it start with different h_0
        # For now it's zero-initialied, but maybe there is something smarter 
        # that can be done.
        
        self.batch_size = batch_size 
        n_layers = 1                
        self.hidden_state = torch.zeros(n_layers, self.batch_size, self.fc3.in_features)
        self.cell_state = torch.zeros(n_layers, self.batch_size, self.fc3.in_features)
            
    def forward(self, x):
        
        self.hidden = (self.hidden_state, self.cell_state)
        x = self.fc1(x)
        x = self.fc2(x)
        
        x = x[None, :, :] # I don't know why you have to add a dimension here
        
        self.hidden, cell_state = self.lstm(x, self.hidden)
        x = self.fc3(self.hidden)        
        x = torch.nn.functional.softmax(x, dim = 1)
        
        return x
    
tensors = torch.tensor([[1., -1.],[2., 15.],[1., -1.],[20., 12.]])
#tensors = torch.tensor([[1., -1.]])
Net = EmbeddingPlusLSTM(2, 8, 5, 10, len(tensors)) 
# 2 is the number of features of an Object
# 8 is the number of features for the embedding
# 5 number of features for the hidden size of the rnn
#10 is the number of output features
# the last one is the batch size
params = list(Net.parameters())

x = Net(tensors)
print(x)
x = Net(tensors)
print(x)

tensor([[[0.2479, 0.2196, 0.2969, 0.2381, 0.2964, 0.2422, 0.2296, 0.2567,
          0.2861, 0.2800],
         [0.2371, 0.3081, 0.1691, 0.2747, 0.1790, 0.2802, 0.2762, 0.2382,
          0.2011, 0.2167],
         [0.2479, 0.2196, 0.2969, 0.2381, 0.2964, 0.2422, 0.2296, 0.2567,
          0.2861, 0.2800],
         [0.2670, 0.2528, 0.2370, 0.2490, 0.2282, 0.2353, 0.2647, 0.2485,
          0.2267, 0.2233]]], grad_fn=<SoftmaxBackward>)
tensor([[[0.2479, 0.2196, 0.2969, 0.2381, 0.2964, 0.2422, 0.2296, 0.2567,
          0.2861, 0.2800],
         [0.2371, 0.3081, 0.1691, 0.2747, 0.1790, 0.2802, 0.2762, 0.2382,
          0.2011, 0.2167],
         [0.2479, 0.2196, 0.2969, 0.2381, 0.2964, 0.2422, 0.2296, 0.2567,
          0.2861, 0.2800],
         [0.2670, 0.2528, 0.2370, 0.2490, 0.2282, 0.2353, 0.2647, 0.2485,
          0.2267, 0.2233]]], grad_fn=<SoftmaxBackward>)


Now you should put the real RL framework.

In [ ]:
Net = EmbeddingPlusLSTM(2, 8, 5, 10) 
# 2 is the number of features of an Object
# 10 is the number of features for the embedding
# 5 number of features for the hidden size of the rnn

params = list(Net.parameters())
#tensors = torch.tensor([[1., -1.],[2., 15.],[1., -1.],[2., 15.]])
#tensors = torch.tensor([[1., -1.]])
#x = Net(tensors)
#print(x)

In [ ]:
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update


for i_counter in len(range(num_instances)):
    num_objs = 10
    Objects = GenerateANewInstance(num_objs)
    total_reward = 0.0
    total_steps = 0

    while True:
        choose action
        observe reward
        total_steps += 1
        if action.Prize == 0 and action.Weight == 0:
            break

    print("Episode done in %d steps, total reward %.2f" % (total_steps, Agent.totoal_reward))    
    stop